# LLM VectorDB 저장
#### w/ FAISS >> Chroma

In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader

from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

import pandas as pd

# VectorDB - FAISS
from langchain_community.vectorstores import Chroma

In [2]:
### 01. CSV 파일에서 문서 로드 ###
loader = CSVLoader('../data/movies_mart_14224.csv', encoding='utf8')
docs = loader.load()
print(f"문서의 수: {len(docs)}")

### 02. pandas로 데이터프레임 칼럼명 가져오기
csv_path = '../data/movies_mart_14224.csv'
df2 = pd.read_csv(csv_path, encoding='utf8')
colnames = df2.columns

문서의 수: 14224


In [3]:
colnames

Index(['asset_id', 'title', 'original_title', 'genre', 'adult', 'runtime',
       'release_year', 'release_month', 'release_day', 'actors', 'director',
       'orgnl_cntry', 'original_language', 'vote_average', 'vote_count',
       'popularity', 'poster_path', 'backdrop_path', 'overview', '분류'],
      dtype='object')

In [4]:
# 'runtime', 'release_year', 'release_month', 'release_day', 'vote_count'정수값으로 변경
# 실수(float) → 정수(Int64) 변환 (NaN 허용)
# df2['runtime'] = df2['runtime'].astype(pd.Int64Dtype())
# df2['release_year'] = df2['release_year'].astype(pd.Int64Dtype())
# df2['release_month'] = df2['release_month'].astype(pd.Int64Dtype())
# df2['release_day'] = df2['release_day'].astype(pd.Int64Dtype())
# df2['vote_count'] = df2['vote_count'].astype(pd.Int64Dtype())

In [5]:
### 03. 메타데이터 추가 ###
docs = []
for _, row in df2.iterrows():
  # 필요한 메타데이터 설정
  metadata = {
    'asset_id': row['asset_id'],
    'title': row['title'],
    'original_title': row['original_title'],
    'genre': row['genre'],
    'adult': row['adult'],
    'runtime': row['runtime'],
    'release_year': row['release_year'],
    'release_month': row['release_month'],
    'release_day': row['release_day'],
    'actors': row['actors'],
    'director': row['director'],
    'orgnl_cntry': row['orgnl_cntry'],
    'original_language': row['original_language'],
    'vote_average': row['vote_average'],
    'vote_count': row['vote_count'],
    'popularity': row['popularity']
  }
  # 각 행의 데이터를 문서로 변환
  doc = Document(
    page_content=str(row.to_dict()),
    metadata=metadata
  )
  docs.append(doc)

print(f"문서의 수: {len(docs)}")
print('[메타데이터 예시]\n', docs[100].metadata)

문서의 수: 14224
[메타데이터 예시]
 {'asset_id': 'cjc|M0376427LSG233188001', 'title': '캣우먼', 'original_title': 'Catwoman', 'genre': '범죄,액션,판타지', 'adult': False, 'runtime': 104.0, 'release_year': 2004.0, 'release_month': 7.0, 'release_day': 22.0, 'actors': '할 베리,벤자민 브렛,랑베르 윌슨,프란시스 콘로이', 'director': '피토프', 'orgnl_cntry': '미국', 'original_language': 'en', 'vote_average': 4.613, 'vote_count': 3327.0, 'popularity': 29.052}


In [6]:
### 04. 데이터 청크 나누기 ###
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1630, chunk_overlap=0
)
splits = text_splitter.split_documents(docs)
print("split된 문서의 수:", len(splits))

split된 문서의 수: 14224


In [7]:
### 05. 임베딩 모델 생성
# https://huggingface.co/ibm-granite/granite-embedding-278m-multilingual
embeddings = HuggingFaceEmbeddings(model_name='ibm-granite/granite-embedding-278m-multilingual')

C:\Users\USER\AppData\Local\Temp\ipykernel_16172\3636801640.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='ibm-granite/granite-embedding-278m-multilingual')
c:\workspaces\LGDXteam2\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
### 06. 벡터스토어 생성 ###
## FAISS
vectorstore = Chroma.from_documents(
  documents=splits,
  embedding=embeddings,
  persist_directory="../db/movies_vectorstore_chroma_1630"
)

print("Chroma 저장 완료!")

Chroma 저장 완료!


In [9]:
# 벡터스토어 로드
new_vector_store = Chroma(persist_directory="../db/movies_vectorstore_chroma_1630",
                          embedding_function=embeddings
                          )

C:\Users\USER\AppData\Local\Temp\ipykernel_16172\1512383776.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  new_vector_store = Chroma(persist_directory="../db/movies_vectorstore_chroma_1630",
